In [2]:
%load_ext autoreload
%autoreload 2

In [20]:
from IPython.display import JSON

In [3]:
import qat

from qat.model.model import QuantumHardwareModel as QuantumCalibrationModel, GateSet
from qat.utils.pydantic import WarnOnExtraFieldsModel

from pydantic import ConfigDict, Field, model_validator
from typing import Dict, List, Tuple, Union

In [46]:
class At(WarnOnExtraFieldsModel):
    x: int

In [47]:
class Bt(WarnOnExtraFieldsModel):
    x: int

In [60]:
class Ct(WarnOnExtraFieldsModel):
    x: int
    id: str

In [61]:
class Dt(WarnOnExtraFieldsModel):
    x: int
    Cs: Dict[str, Ct]

In [62]:
class Outer(WarnOnExtraFieldsModel):
    A: List[At]
    B: List[Bt]
    C: List[Ct]
    D: List[Dt]

In [69]:
A = [At(x=i) for i in range(5)]
B = [Bt(x=100+i) for i in range(5)]
C = [Ct(x=100+i, id=str(i)) for i in range(2)]
D = [Dt(x=100+i, Cs={c.id: c for c in C[0:1]}) for i in range(5)]

In [70]:
O = Outer(A=[], B=[],C=C,D=D)

In [71]:
JSON(O.model_dump())

<IPython.core.display.JSON object>